## Vistual inertia scheduling

In [1]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
import torch

logger = logging.getLogger(__name__)

# from opf import dcopf
from visopf import vis1

import andes.interop.pandapower as adpp

## Main - load IEEE 39 and external parameters


### load case from andes excel

In [2]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

### load norm parameter

In [3]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }


### load nn parameter

In [4]:
# frequency nadir prediction network
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [5]:
# vsg peak power prediction network
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [6]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [7]:
ss = vis1(norm=norm, nn=nn, dpe=0.067, rocof_lim=0.01, nadir_lim=0.01)

Restricted license - for non-production use only - expires 2023-10-25


In [8]:
# define typeII gen (VSG inverter)
vsg_ieee14 = ['PV_6', 'PV_7']
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']

ss.from_andes(ssa, vsg_ieee39, Sbase=1000)

In [9]:
ss.build()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.


In [10]:
# revise gen cost
ss.cost['c1'].iloc[0] = 1
ss.cost['c1'].iloc[5] = 0.5
ss.cost['c1'].iloc[9] = 1.5

ss.update_dict()
ss.costdict

{'PV_1': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_2': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_3': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_4': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_5': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_6': {'c2': 0.0, 'c1': 0.5, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_7': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_8': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_9': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cr': 0.0, 'cru': 0.0, 'crd': 0.0},
 'Slack_10': {'c2': 0.0,
  'c1': 1.5,
  'c0': 0.0,
  'cr': 0.0,
  'cru': 0.0,
  'crd': 0.0}}

In [11]:
ss.get_res()

Successfully build var.
Successfully build obj.
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Successfully build cons.
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 264 rows, 166 columns and 2790 nonzeros
Model fingerprint: 0xe49cf77e
Variable types: 102 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e-07, 1e+02]
  Objective range  [5e-01, 2e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [5e-03, 1e+02]
Presolve removed 183 rows and 114 columns
Presolve time: 0.00s
Presolved: 81 rows, 52 columns, 613 nonzeros
Variable types: 32 continuous, 20 integer (20 binary)

Root relaxation: objective 5.606400e+01, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*   

(        gen      Sn         pg  pru  prd
 0      PV_1  10.400  11.714575  0.0  0.0
 1      PV_2   8.360   2.886426  0.0  0.0
 2      PV_3   8.437   8.000000  0.0  0.0
 3      PV_4  11.748   5.851096  0.0  0.0
 4      PV_5  10.802   3.800000  0.0  0.0
 5      PV_6  10.857   8.000000  0.0  0.0
 6      PV_7  10.252   7.000000  0.0  0.0
 7      PV_8   9.702   3.501478  0.0  0.0
 8      PV_9  16.841   4.810425  0.0  0.0
 9  Slack_10  11.990   3.000000  0.0  0.0,
     gen  Mvsg  Dvsg
 0  PV_1   5.0   0.0
 1  PV_6   5.0   0.0
 2  PV_8   5.0   0.0
 3  PV_9   5.0   0.0,
 {'Msys': 6.930391632467617,
  'Dsys': 0.5893050315845287,
  'Rsys': 20.933442043222,
  'Fsys': 10.039026364939438})